# 04_model（Decision Tree モデル）
## 【目的】
EDA・特徴量設計を通じて作成したデータ（`employee_with_features.csv`）を用い、離職リスクを予測できるモデルを構築する。
これにより、どんな要素が離職に影響しているかを明確化「早期発見」「改善の優先順位づけ」が可能になる

## 【理由】

### ① 決定木は説明しやすい  
→ 「どんな条件で離職と判断されるか」がルールとして可視化される。  
→ 説明に向いている。

### ② カテゴリ変数に強い  
性別・年代・部署など、カテゴリデータを多く含む人事データに適している。

### ③ 重要度（Feature Importance）を算出できる  
→ モデルがどの特徴を重視しているかを数値で把握できる。  
→ 改善施策の優先順位を客観的に決められる。


## 結果

##### 適合率（precision）
0(在職者)：在職と予測したうち、99%正解(1%は誤報)  
1(離職者): 離職と予測したうち、43%正解(57％は誤報)

##### 再現率（recall）
0(在職者):在職者(9634人)のうち、97％は見逃さず予測できた  
1(離職者):離職者(297人)のうち、75%は見逃さず予測できた  

##### F1スコア（precisionとrecallのバランス）
0(在職者):0.98 離職者の97%を拾えており、誤報も1%  
1(離職者):0.54 離職者75%拾えており、誤報も一定程度抑えられている。


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier


In [2]:
# 1 データ読み込み
df = pd.read_csv("data/processed/employee_with_features.csv")

In [ ]:
# 2️ 目的変数とリーク列処理

# 「ACTIVE」「TERMINATED」は文字列なので、そのままだとエラーになるため0/1の数値に変換する。
# STATUSが"TERMINATED"なら1（True）、それ以外は0（False）とする。
df["target"] = (df["STATUS"]  == "TERMINATED").astype(int)

# リーク列,予測に不要な列を抽出
drop_col = [
    "STATUS", "terminationdate_key", "termreason_desc", "termtype_desc",
    "EmployeeID", "gender_full", "orighiredate_key", "birthdate_key",
    "recorddate_key", "attrition_flag"
]

#　説明変数と目的変数を作成
X = df.drop(columns=drop_col + ["target"])
y = df["target"]

In [ ]:
# 3️ 学習データ分割（再現性のためrandom_state固定・目的変数の比率維持）
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [5]:
# 数値列を確認
X.select_dtypes(include=["number"]).columns

Index(['age', 'length_of_service', 'store_name', 'STATUS_YEAR',
       'is_young_female', 'is_senior_male', 'relative_service',
       'age_group_encoded'],
      dtype='object')

In [6]:
# 4 前処理

# 連続値は、スケーリング（平均0,分散1に標準化）して、値のスケールを揃える(例：勤続年数が「15」と「1」だと数字がデカすぎて他より目立ってしまう。「標準化」で均等な重み付けに直す。)
scale_cols = ["age", "length_of_service", "relative_service"]
# 数値だけど実はカテゴリのデータ（0/1o r順序）はスケーリングしない。(age_group_encodedは、順序カテゴリ)
num_cols = ["is_young_female", "is_senior_male", "age_group_encoded"]
#文字（カテゴリ）データをまとめて取得
cat_cols = X.select_dtypes(exclude=["number"]).columns

In [7]:
#　ColumnTransformerを使用。複数の列グループに対して、異なる前処理を“並列”に適用する
#【理由】今回のデータにおいては欠損値なしのため欠損値補完は不要。また、決定木のため標準化も不要ではあるが、将来の欠損対応,線形モデル切替時に役立つことを想定し作成。

preprocess = ColumnTransformer([
    #数値データを処理
    ("scale", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]), scale_cols),
    ("num", SimpleImputer(strategy="most_frequent"), num_cols),
    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))#学習時に知らないカテゴリが出てもそのカテゴリは「全部0（新カテゴリ扱い）」として処理
    ]), cat_cols)
])


In [8]:
# 5️ pipelineで前処理からモデル作成を一連の流れとしてまとめる
# 【理由】同じ処理を検証できる再現性、簡単にモデルの差し替え可能。

model = Pipeline([
    ("preprocess", preprocess),
    ("clf", DecisionTreeClassifier(max_depth=5, random_state=0, class_weight="balanced"))
])

In [12]:
# # 6️ 学習
model.fit(X_train, y_train)

,steps,"[('preprocess', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('scale', ...), ('num', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [14]:
# ① 予測
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:,1]



In [26]:
# y_pred(0/1)だと閾値0.5を切った結果しか見えないが、y_pred_probaを使うことで、どのくらい確信しているかまでわかる。

y_pred_proba[:5]

# 1人目の離職確率 14%,
# 2人目の離職確率 23%,
# 3人目の離職確率 14%,
# 4人目の離職確率 14%,
# 5人目の離職確率 14%,

array([0.14058505, 0.23114166, 0.14058505, 0.14058505, 0.14058505])

In [16]:
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# ② 精度（まずざっくり確認）
print("Accuracy:", accuracy_score(y_test, y_pred))
print

Accuracy: 0.9624408418084786


In [31]:
print("AUC:", roc_auc_score(y_test, y_pred_proba))


AUC: 0.8944078876090501


##### 適合率（precision）
0(在職者)：在職と予測したうち、99%正解(1%は誤報)  
1(離職者): 離職と予測したうち、43%正解(57％は誤報)

##### 再現率（recall）
0(在職者):在職者(9634人)のうち、97％は見逃さず予測できた  
1(離職者):離職者(297人)のうち、75%は見逃さず予測できた  

##### F1スコア（precisionとrecallのバランス）
0(在職者):0.98 離職者の97%を拾えており、誤報も1%  
1(離職者):0.54 離職者75%拾えており、誤報も一定程度抑えられている。

In [28]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.99      0.97      0.98      9634
           1       0.43      0.75      0.54       297

    accuracy                           0.96      9931
   macro avg       0.71      0.86      0.76      9931
weighted avg       0.98      0.96      0.97      9931



In [29]:
# ④ 過学習チェック
print("Train acc:", model.score(X_train, y_train))
print("Test acc:", model.score(X_test, y_test))


Train acc: 0.9573284326066159
Test acc: 0.9624408418084786


In [32]:
import pickle 


#　前処理を保存 (05_evaluation.ipynb) 
with open("models/preprocess.pkl", "wb") as f:
    pickle.dump(model.named_steps["preprocess"], f)
print("学習したデータをもとに作成した前処理を保存しました！（models/01_DecisionTree.pkl）")

#学習後にpreprocessを保存することで、テストデータや新しいデータを学習して覚えたルールで変換できない .transform()
# # SimpleImputer → 学習データの中央値を記憶
# StandardScaler → 学習データの平均・標準偏差を記憶
# OneHotEncoder → 学習データに登場したカテゴリを記憶


#　モデル全体を保存
with open("models/01_DecisionTree.pkl","wb") as f:
    pickle.dump(model,f)

print("モデルを保存しました！（models/01_DecisionTree.pkl）")


学習したデータをもとに作成した前処理を保存しました！（models/01_DecisionTree.pkl）
モデルを保存しました！（models/01_DecisionTree.pkl）
